# All solutions made using Qiskit 1.2.4.

In [4]:
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit.visualization import array_to_latex
from qiskit import QuantumCircuit

### Exercise 1: Find state of single qubit for which probability of measuring $\ket{0}$ is 4 times larger than measuring $\ket{1}$.

In [5]:
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import StatevectorSampler

initial_state = Statevector([np.sqrt(4)/np.sqrt(5), 1/np.sqrt(5)])

array_to_latex(initial_state)

<IPython.core.display.Latex object>

Let's (semi)proof it:

In [6]:
qc = QuantumCircuit(1, 1)
qc.initialize(initial_state, [0])
qc.measure_all()

sampler = StatevectorSampler()

num_shots = 1000000
pm = generate_preset_pass_manager(optimization_level=1)
isa_circuit = pm.run(qc)
result = sampler.run([isa_circuit], shots=num_shots).result()
data_pub = result[0].data
counts = data_pub.meas.get_counts()

for i in counts:
    counts[i] = float(counts[i]/num_shots)

print(f"The probabilities are: {counts}")

The probabilities are: {'0': 0.800226, '1': 0.199774}


### Exercise 2: normalize vectors [1, -i, 1, 0] and [5, 2i, 1, 1]

In [7]:
v1 = np.array([[1],[-1.j],[1],[0]])
norm = np.linalg.norm(v1)
normalized_v1= v1/norm
array_to_latex(normalized_v1)

<IPython.core.display.Latex object>

In [8]:
v2 = np.array([[5],[2.j],[1],[1]])
norm = np.linalg.norm(v2)
normalized_v2 = v2/norm
array_to_latex(normalized_v2)

<IPython.core.display.Latex object>

### Exercise 3: What is the result of applying of the following operator on the 2-qubit bell state:
$
\begin{bmatrix} 
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 1 & 0 \\
0 & 1 & 0 & 0
\end{bmatrix}
$

In [9]:
qc = QuantumCircuit(2,2)
qc.h(0)
qc.cx(0,1)
bell_state = Statevector.from_instruction(qc)
array_to_latex(bell_state)

<IPython.core.display.Latex object>

In [10]:
from qiskit.quantum_info import Operator
op = Operator(np.array([[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0]]))
final_state = bell_state.evolve(op)
array_to_latex(final_state)

<IPython.core.display.Latex object>

### Exercise 4 - find the matrix representation of 3 qubit state construsted from 2-qubit Bell's state and 3rd qubit with X gate

In [11]:
from qiskit import *
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit import QuantumCircuit

In [12]:
qc2 = QuantumCircuit(3,3)
qc2.h(0)
qc2.cx(0, 1)
qc2.x(2)

op2 = Operator.from_circuit(qc2)
array_to_latex(op2)

<IPython.core.display.Latex object>

### Exercise 5: Using Estimator primitive, calculate expectation value of the operator from Exercise 4 in the 3-qubit GHZ state

First, let's check if the observable  is hermitian. Only for such observables Estimator primitive can calculate expectation values.\
We can for example calculate eigenvalues to check if they are real:

In [13]:
from numpy import linalg as la
array_to_latex(la.eigvals(op2))

<IPython.core.display.Latex object>

...or we can check this directly, for example in such way:

In [14]:
from scipy.linalg import ishermitian

ishermitian(op2.data)

False

Let's check how does the matrix look like in Pauli form:

In [16]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_operator(op2)
print(observable)

SparsePauliOp(['XII', 'XIX', 'XIY', 'XIZ', 'XXI', 'XXX', 'XXY', 'XXZ'],
              coeffs=[ 0.35355339+0.j        ,  0.35355339+0.j        ,  0.        +0.35355339j,
  0.35355339+0.j        , -0.35355339+0.j        ,  0.35355339+0.j        ,
 -0.        -0.35355339j,  0.35355339+0.j        ])


We see that not all coefficients are real - such observables cannot be used with Estimator (!!!)\
Let's remove the imaginary coefficients to get similar, although different observable - this is all we can do here :)

In [17]:
observable_modified = SparsePauliOp(['XII', 'XIX', 'XIY', 'XIZ', 'XXI', 'XXX', 'XXY', 'XXZ'],
    coeffs=[0.35355339,  0.35355339,  0, 0.35355339, -0.35355339,  0.35355339, 0,  0.35355339])
array_to_latex(observable_modified)

<IPython.core.display.Latex object>

In [20]:
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

# Let's define our state (as a result of the following circuit):
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0, 1)
circuit.cx(1, 2)

observable = observable_modified

#Uncomment next line to check what happens with original observable:
#observable = SparsePauliOp.from_operator(op2)

estimator = StatevectorEstimator()

job = estimator.run([(circuit, observable)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.35355338999999997
 > Metadata: {'target_precision': 0.0, 'circuit_metadata': {}}


### Exercise 6: Use estimator to calculate expectation value of observable $\hat{O}=2II-2XX+3YY-3ZZ$ in Bell's state

In [21]:
# Let's define our state (as a result of the following circuit):
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)

observable_o = SparsePauliOp(['II', 'XX', 'YY', 'ZZ'], coeffs=[2,  -2,  3, -3])
 
estimator = StatevectorEstimator()

job = estimator.run([(circuit, observable_o)])
result = job.result()

print(f"Expectation value: {result[0].data.evs}")

Expectation value: -5.999999999999998


Without Estimator, this could be done much simpler:

In [22]:
print(Statevector.from_instruction(circuit).expectation_value(observable_o))

(-5.999999999999998+0j)


### Exercise 7: Find matrix representation of the $\hat{O}$ operator

In [23]:
array_to_latex(observable_o)

<IPython.core.display.Latex object>

In [24]:
array_to_latex(la.eigvals(observable_o))

<IPython.core.display.Latex object>